Подключим необходимые для работы библиотеки. Будем использовать word2vec преобразование слов.

In [1]:
from gensim.models import Word2Vec
import joblib
import json
from nltk.corpus import stopwords
import numpy as np
from pymorphy3 import MorphAnalyzer
import re
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
import sqlite3
from tensorflow import keras

2023-05-26 23:05:57.486174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Напишем функцию для токенизации и лемматизации слов, а также удаления стопслов из документа (поста, описания группы и любого другого текстового параметра). Функция токенизирует документ, если в нем содержалось более 2 значащих слов.

In [2]:
patterns = "[«»!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru_en = stopwords.words("russian") + stopwords.words("english")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = doc.lower()
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        token = token.strip()
        if token and token not in stopwords_ru_en:
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

Извлечем тексты всех постов из датасета, описания групп, их названия и статусы для обучения модели word2vec, разделим данные на обучающий и тестовый датасеты.

In [3]:
db = sqlite3.connect('groups.db')  # путь к датасету
cursor_obj = db.cursor()
cursor_obj.execute("select * from groups")
rows = cursor_obj.fetchall()
db.close()

train_data, test_data = train_test_split(rows, test_size=0.25)

Соберем все тексты, которые содержатся в нашем датасете, в единый список документов

In [4]:
all_texts = []
for group in rows:
    name = lemmatize(group[1])
    if name is not None:
        all_texts.append(name)
    status = lemmatize(group[2])
    if status is not None:
        all_texts.append(status)
    description = lemmatize(group[3])
    if description is not None:
        all_texts.append(description)
    posts = json.loads(group[12])
    for post in posts:
        prep_post = lemmatize(post['text'])
        if prep_post is not None:
            all_texts.append(prep_post)

Обучим w2v модель на наших текстах.

In [5]:
w2v_model = Word2Vec(
    all_texts,
    min_count=10,
    window=2,
    vector_size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

w2v_model.train(all_texts, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)

(2757601, 10085300)

Запишем вспомогательные функции для обработки текстов. Первая вычисляет сходство текста с какой-либо тематикой. Вторая функция вычисляет сходство текста со всеми тематиками и формирует вектор для текста.

In [6]:
def get_text_similarity(tokenized_text, theme_word):
    text_similarity = 0.
    used_words = 0
    for word in tokenized_text:
        if word in w2v_model.wv.key_to_index:
            text_similarity += w2v_model.wv.similarity(word, theme_word)
            used_words += 1
    if used_words != 0:
        return text_similarity / used_words
    return 0.

def get_themes_characteristics(tokenized_text):
    return [
        get_text_similarity(tokenized_text, 'музыка'),
        get_text_similarity(tokenized_text, 'путешествие'),
        get_text_similarity(tokenized_text, 'программирование'),
        get_text_similarity(tokenized_text, 'мем')
    ]

Создадим функцию для преобразования информации о группе в вектор. В векторе содержатся данные о названии группы, статусе и описании, численные значения о группе (число подписчиков и т.п.), а также данные о 15 постах группы (текст поста, число лайков, репостов, прикрепленных фото и т.п.). Также написана функция для получения только текстовых признаков группы.

In [7]:
def get_vectorized_group_info(group_info):
    group_vector = []
    if lemmatize(group_info[1]) is not None:
        group_vector.extend(get_themes_characteristics(lemmatize(group_info[1])))
    else:
        group_vector.extend([0.] * 4)
    if lemmatize(group_info[2]) is not None:
        group_vector.extend(get_themes_characteristics(lemmatize(group_info[2])))
    else:
        group_vector.extend([0.] * 4)
    if lemmatize(group_info[3]) is not None:
        group_vector.extend(get_themes_characteristics(lemmatize(group_info[3])))
    else:
        group_vector.extend([0.] * 4)
    group_vector.extend(group_info[6:11])
    posts = json.loads(group_info[12])
    for post in posts:
        if lemmatize(post['text']) is not None:
            group_vector.extend(get_themes_characteristics(lemmatize(post['text'])))
        else:
            group_vector.extend([0.] * 4)
        group_vector.extend([post['likes'], post['reposts'], post['photos_number'], post['music_number'], 
                        post['video_number'], post['links_number'], post['docs_number']])
    while len(group_vector) < 3 * 4 + 5 + 15 * 11:
        group_vector.extend([0.] * 11)
    
    return group_vector


def get_vectorized_group_info_only_texts(group_info):
    group_vector = []
    if lemmatize(group_info[1]) is not None:
        group_vector.extend(get_themes_characteristics(lemmatize(group_info[1])))
    else:
        group_vector.extend([0.] * 4)
    if lemmatize(group_info[2]) is not None:
        group_vector.extend(get_themes_characteristics(lemmatize(group_info[2])))
    else:
        group_vector.extend([0.] * 4)
    if lemmatize(group_info[3]) is not None:
        group_vector.extend(get_themes_characteristics(lemmatize(group_info[3])))
    else:
        group_vector.extend([0.] * 4)
    posts = json.loads(group_info[12])
    for post in posts:
        if lemmatize(post['text']) is not None:
            group_vector.extend(get_themes_characteristics(lemmatize(post['text'])))
        else:
            group_vector.extend([0.] * 4)
    while len(group_vector) < 3 * 4 + 15 * 4:
        group_vector.extend([0.] * 4)
    
    return group_vector

Сформируем векторы входных и выходных данных для моделей, используя написанные функции. Создадим следующие наборы: датасет со всеми возможными входными данными, датасет без информации о постах и датасет, содержащий только информацию о текстовых данных.

In [8]:
themes = {
    'Музыка': 1,
    'Путешествия': 2,
    'Программирование': 3,
    'Мемы': 4,
    'Другое': 0
}
themes_inverse = {
    1: 'Музыка',
    2: 'Путешествия',
    3: 'Программирование',
    4: 'Мемы',
    0: 'Другое'
}

train_input_full = []
train_input_no_posts = []
train_input_only_texts = []
train_output = []
for element in train_data:
    train_input_full.append(get_vectorized_group_info(element))
    train_input_no_posts.append(train_input_full[-1][:17])
    train_input_only_texts.append(get_vectorized_group_info_only_texts(element))
    train_output.append(themes[element[13]])
    
test_input_full = []
test_input_no_posts = []
test_input_only_texts = []
test_output = []
for element in test_data:
    test_input_full.append(get_vectorized_group_info(element))
    test_input_no_posts.append(test_input_full[-1][:17])
    test_input_only_texts.append(get_vectorized_group_info_only_texts(element))
    test_output.append(themes[element[13]])

Обучим классификатор RidgeClassifier на полных данных и посчитаем метрики для оценки модели:

In [9]:
model_ridge_full = linear_model.RidgeClassifier()
model_ridge_full.fit(train_input_full, train_output)

predicted_test_ridge = model_ridge_full.predict(test_input_full)
print(metrics.classification_report(predicted_test_ridge, test_output))

              precision    recall  f1-score   support

           0       0.40      0.15      0.22        13
           1       0.58      1.00      0.74         7
           2       0.50      0.62      0.56         8
           3       0.58      0.64      0.61        11
           4       0.36      0.36      0.36        11

    accuracy                           0.50        50
   macro avg       0.49      0.56      0.50        50
weighted avg       0.47      0.50      0.46        50



Для данных, не использующих информацию о постах:

In [10]:
model_ridge_no_posts = linear_model.RidgeClassifier()
model_ridge_no_posts.fit(train_input_no_posts, train_output)

predicted_test_no_posts = model_ridge_no_posts.predict(test_input_no_posts)
print(metrics.classification_report(predicted_test_no_posts, test_output))

              precision    recall  f1-score   support

           0       0.20      0.10      0.13        10
           1       0.75      0.64      0.69        14
           2       0.90      0.82      0.86        11
           3       0.67      1.00      0.80         8
           4       0.36      0.57      0.44         7

    accuracy                           0.62        50
   macro avg       0.58      0.63      0.59        50
weighted avg       0.61      0.62      0.60        50



Для данных, использующих только текстовую информацию:

In [11]:
model_ridge_only_texts = linear_model.RidgeClassifier()
model_ridge_only_texts.fit(train_input_only_texts, train_output)

predicted_test_only_texts = model_ridge_only_texts.predict(test_input_only_texts)
print(metrics.classification_report(predicted_test_only_texts, test_output))

              precision    recall  f1-score   support

           0       0.40      0.25      0.31         8
           1       0.58      0.78      0.67         9
           2       0.90      1.00      0.95         9
           3       0.92      0.92      0.92        12
           4       0.64      0.58      0.61        12

    accuracy                           0.72        50
   macro avg       0.69      0.71      0.69        50
weighted avg       0.70      0.72      0.71        50



Обучим модели многослойного перцептрона на наших данных, чтобы сравнить результат ее работы с RidgeClassifier:

In [12]:
np_train_input_full = np.array(train_input_full)
np_test_input_full = np.array(test_input_full)

train_output_onehot = keras.utils.to_categorical(np.array(train_output))

model_seq_full = keras.Sequential([
        keras.layers.Dense(np_train_input_full.shape[1], activation='elu'),
        keras.layers.Dense(100, activation='elu'),
        keras.layers.Dense(60, activation='elu'),
        keras.layers.Dense(train_output_onehot.shape[1], activation='softmax')
    ])
model_seq_full.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
history_seq_full = model_seq_full.fit(np_train_input_full, train_output_onehot, epochs=100, verbose=0)

predicted_seq_test_full = model_seq_full.predict(np_test_input_full)
predicted_res_test_full = []
for elem in predicted_seq_test_full:
    max_index = 0
    for i in range(5):
        if elem[i] > elem[max_index]:
            max_index = i
    predicted_res_test_full.append(max_index)
print(metrics.classification_report(predicted_res_test_full, test_output))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.75      0.26      0.39        34
           2       0.20      0.25      0.22         8
           3       0.33      0.80      0.47         5
           4       0.00      0.00      0.00         1

    accuracy                           0.30        50
   macro avg       0.26      0.26      0.22        50
weighted avg       0.58      0.30      0.35        50



In [13]:
np_train_input_no_posts = np.array(train_input_no_posts)
np_test_input_no_posts = np.array(test_input_no_posts)

model_seq_no_posts = keras.Sequential([
        keras.layers.Dense(np_train_input_no_posts.shape[1], activation='elu'),
        keras.layers.Dense(100, activation='elu'),
        keras.layers.Dense(60, activation='elu'),
        keras.layers.Dense(train_output_onehot.shape[1], activation='softmax')
    ])
model_seq_no_posts.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
history_seq_no_posts = model_seq_no_posts.fit(np_train_input_no_posts, train_output_onehot, epochs=100, verbose=0)

predicted_seq_test_no_posts = model_seq_no_posts.predict(np_test_input_no_posts)
predicted_res_test_no_posts = []
for elem in predicted_seq_test_no_posts:
    max_index = 0
    for i in range(5):
        if elem[i] > elem[max_index]:
            max_index = i
    predicted_res_test_no_posts.append(max_index)
print(metrics.classification_report(predicted_res_test_no_posts, test_output))

2/2 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.80      0.12      0.21        33
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.67      0.47      0.55        17
           4       0.00      0.00      0.00         0

    accuracy                           0.24        50
   macro avg       0.29      0.12      0.15        50
weighted avg       0.75      0.24      0.33        50



/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
np_train_input_only_texts = np.array(train_input_only_texts)
np_test_input_only_texts = np.array(test_input_only_texts)

model_seq_only_texts = keras.Sequential([
        keras.layers.Dense(np_train_input_only_texts.shape[1], activation='elu'),
        keras.layers.Dense(100, activation='elu'),
        keras.layers.Dense(60, activation='elu'),
        keras.layers.Dense(train_output_onehot.shape[1], activation='softmax')
    ])
model_seq_only_texts.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
history_seq_only_texts = model_seq_only_texts.fit(np_train_input_only_texts, train_output_onehot, epochs=100, verbose=0)

predicted_seq_test_only_texts = model_seq_only_texts.predict(np_test_input_only_texts)
predicted_res_test_only_texts = []
for elem in predicted_seq_test_only_texts:
    max_index = 0
    for i in range(5):
        if elem[i] > elem[max_index]:
            max_index = i
    predicted_res_test_only_texts.append(max_index)
print(metrics.classification_report(predicted_res_test_only_texts, test_output))

2/2 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.60      0.25      0.35        12
           1       0.67      0.80      0.73        10
           2       0.90      0.90      0.90        10
           3       0.83      0.91      0.87        11
           4       0.55      0.86      0.67         7

    accuracy                           0.72        50
   macro avg       0.71      0.74      0.70        50
weighted avg       0.72      0.72      0.69        50



Можно сделать вывод, что в первых двух моделях слишком много данных, к тому же они плохо нормированы (характеристики текстов меньше 1, в то время как число лайков/комментариев имеет значения в тысячи, а то и сотни тысяч). Третий вариант с использованием только текстовых данных наиболее работающий, имеет неплохие показатели в обоих случаях. Многослойный перцептрон работает лучше обычного линейного классификатора.

Сохраним обученные модели для дальнейшего использования:

In [15]:
joblib.dump(model_ridge_full, 'model_ridge_full.pkl')
joblib.dump(model_ridge_no_posts, 'model_ridge_no_posts.pkl')
joblib.dump(model_ridge_only_texts, 'model_ridge_only_texts.pkl')
model_seq_full.save('model_seq_full.h5')
model_seq_no_posts.save('model_seq_no_posts.h5')
model_seq_only_texts.save('model_seq_only_texts.h5')